In [1]:
import requests
import pandas as pd
import os
import time

In [2]:
#  请求地址前缀
req_url_pref = "http://api.tianditu.gov.cn/geocoder?"
amp_api_key = 'xxx'

def get_poi_from_amap():
    """
    从天地图API下载地理编码
    """
    global result, count, response, data
    response = requests.get(req_url_pref, params = rep_params)
    data = response.json()  # 返回字典数据 dict
    if data['status'] == '0':
        lng = float(data["location"]["lon"])
        lat = float(data["location"]["lat"])
        busi_data = [
            {
                "input_name": amp_api_name,
                "input_address": amp_api_address,
                "gcj2000_lng": lng,
                "gcj2000_lat": lat
            }
        ]
        
    else:
        rep_params1 = {
        "tk": amp_api_key,
        "ds": {
            "{'keyWord':" + '"' + amp_api_name + '"' + "}", 
        }
        }
        response1 = requests.get(req_url_pref, params = rep_params1)
        data = response1.json()
        if data['status'] == '0':
                lng = float(data["location"]["lon"])
                lat = float(data["location"]["lat"])
                busi_data = [
                    {
                        "input_name": amp_api_name,
                        "input_address": amp_api_address,
                        "gcj2000_lng": lng,
                        "gcj2000_lat": lat
                        }
                ]
        else:
            busi_data = [
        {
            "input_name": amp_api_name,
            "input_address": amp_api_address,
            "gcj2000_lng": 'Unknow',
            "gcj2000_lat": 'Unknow'
        }
    ]
            count += 1
            print(count, ":", amp_api_name, " | ", amp_api_address, " | 未能在地图中找到位置")
 
    df = pd.DataFrame(busi_data)
    result = pd.concat([result, df], axis = 0, ignore_index = True)   # 将每次i结果union在一块（列方面追加）

In [3]:
amp_api_data = pd.read_csv('../metadata/cases_address_data_2019_2022.csv')

In [4]:
amp_api_data = amp_api_data.loc[:, ['ID', 'address']]

In [5]:
result = pd.DataFrame()  # 初始化
count = 0

# 遍历地址
# sleep(0.1) -> sleep(2) response: <Response [418]>
for i in range(len(amp_api_data.values)):     
    amp_api_name = amp_api_data.values[i][0]
    amp_api_address = amp_api_data.values[i][1]
    rep_params = {
        'ds': 
            "{'keyWord':"+'"'+ amp_api_address +'"'+"}", 
        'tk': amp_api_key
    }
    get_poi_from_amap()
    if (i + 1) % 50 == 0:
        print("进度：[", i + 1, "/", len(amp_api_data), "]", sep = '')
    time.sleep(3)

进度：[50/2311]
进度：[100/2311]
进度：[150/2311]
进度：[200/2311]
进度：[250/2311]
进度：[300/2311]
进度：[350/2311]
进度：[400/2311]
进度：[450/2311]
进度：[500/2311]
进度：[550/2311]
进度：[600/2311]
进度：[650/2311]
进度：[700/2311]
进度：[750/2311]
进度：[800/2311]
进度：[850/2311]
进度：[900/2311]
进度：[950/2311]
进度：[1000/2311]
进度：[1050/2311]
进度：[1100/2311]
进度：[1150/2311]
进度：[1200/2311]
进度：[1250/2311]
进度：[1300/2311]
进度：[1350/2311]
进度：[1400/2311]
进度：[1500/2311]
进度：[1550/2311]
进度：[1600/2311]
进度：[1650/2311]
进度：[1700/2311]
进度：[1750/2311]
进度：[1800/2311]
进度：[1850/2311]
进度：[1900/2311]
进度：[1950/2311]
进度：[2000/2311]
进度：[2050/2311]
进度：[2100/2311]
进度：[2150/2311]
进度：[2200/2311]
进度：[2250/2311]
进度：[2300/2311]


In [6]:
result.to_csv('../metadata/cases_address_data_2019_2022_with_location.csv', index = False)